In [1]:
from fastai.vision import *
import pandas as pd
from PIL import Image
from matplotlib.cbook import flatten
from matplotlib.pyplot import imshow

%reload_ext autoreload
%autoreload 2
%matplotlib inline

## submission file

First of all let's look at the submission file:

- we have scores for each test image (`10,357` of them) and for each class (`120` classes);
- those scores are **probabilities** - sum of each row is equal to `1`;

In [2]:
path = Path.home() / 'data/db_kaggle'

In [10]:
path_test = path / 'test'

In [6]:
list(path.glob('*'))

[PosixPath('/Users/ilyarudyak/data/db_kaggle/labels.csv'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/.DS_Store'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/temp.txt'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/train'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/sample_submission.csv')]

In [7]:
df = pd.read_csv(path / 'sample_submission.csv')

In [37]:
# as we can see ids are in sorted order
df.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [9]:
df.shape

(10357, 121)

In [14]:
sorted(list(path_test.glob('*')))[:5]

[PosixPath('/Users/ilyarudyak/data/db_kaggle/test/000621fb3cbb32d8935728e48679680e.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/00102ee9d8eb90812350685311fe5890.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/0012a730dfa437f5f3613fb75efcd4ce.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/001510bc8570bbeee98c8d80c8a95ec1.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/001a5f3114548acdefa3d4da05474c2e.jpg')]

In [32]:
list(path_test.glob('*'))[:5]

[PosixPath('/Users/ilyarudyak/data/db_kaggle/test/266490a2b053a4a3bf68e779f28fb5a9.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/dae9fd18836d8df20989ef0101771d4b.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/079a40f22b0fad9a66f76797b62bb930.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/ab8462f9a501d362041d47582fddc9ad.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/8895089b432e020cc42e6262b5a1c2dd.jpg')]

In [12]:
len(list(path_test.glob('*')))

10357

In [26]:
! ls /Users/ilyarudyak/data/db_kaggle/test/ | wc -l

   10357


In [19]:
df.iloc[0, 1:].sum()

0.9999999999999973

## data preparation

In [35]:
bs = 8

In [36]:
data = ImageDataBunch.from_csv(path=path, 
                               folder='train', 
                               csv_labels='labels.csv', 
                               ds_tfms=get_transforms(),
                               suffix='.jpg',
                               test=path_test,
                               size=224,
                               bs=bs,
                               num_workers=0).normalize(imagenet_stats)

First of all let's check that we have test dataloader and dataset. And it looks like that `data.test_ds` has correct number of images (`10357`). Very important that items in the test dataset are **not** sorted. They are in the same order that we get from `glob()`.

In [24]:
len(data.test_dl)

1295

In [25]:
1295 * 8

10360

In [29]:
len(data.test_ds)

10357

In [30]:
len(data.test_ds.items)

10357

In [33]:
list(data.test_ds.items[:5])

[PosixPath('/Users/ilyarudyak/data/db_kaggle/test/266490a2b053a4a3bf68e779f28fb5a9.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/dae9fd18836d8df20989ef0101771d4b.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/079a40f22b0fad9a66f76797b62bb930.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/ab8462f9a501d362041d47582fddc9ad.jpg'),
 PosixPath('/Users/ilyarudyak/data/db_kaggle/test/8895089b432e020cc42e6262b5a1c2dd.jpg')]

Let's build a function to get the numeric part from these paths.

In [47]:
def get_numeric_part(file_path, path_test):
    low = len(str(path_test)) + 1
    return str(file_path)[low:-4]

In [39]:
fp = data.test_ds.items[0]

In [49]:
get_numeric_part(fp, path_test)

'266490a2b053a4a3bf68e779f28fb5a9'

In [ ]:
## predictions

Let's try to get predictions from the model.

In [ ]:
preds, y = learn.get_preds(DatasetType.Test)